In [1]:
import sys
sys.path.insert(0,'/home/berhe/Desktop/Thesis_git/TLP_thesis/codes/')
import Segmantation_lib as est
import Subtitle_truthvalue as stv
import evaluationMetrics as em
import pandas as pd
import os
from pyannote.metrics.diarization import DiarizationCoverage
import numpy as np

Subtitle files and Scene boundries files directory

In [2]:
scenes_dir='/home/berhe/Desktop/Thesis_git/TLP_thesis/Scenes/'
subtitle_dir='/home/berhe/Desktop/Thesis_git/TLP_thesis/subtitles/'

subtitleFiles=[]
for fl in os.listdir(subtitle_dir):
    if '.en.srt' in fl:
       subtitleFiles.append(fl)

scenesegementedFiles=[]
for fl in os.listdir(scenes_dir):
    if 'Episode' in fl:
       scenesegementedFiles.append(fl)
    
subtitleFiles.sort()
scenesegementedFiles.sort()

In [3]:
threads='/home/berhe/Desktop/Thesis_git/TLP_thesis/threadsShot/thread_output5_15_15.json'
shotSeq, shotBondry, _=est.preprocessingThreads(threads=threads)

In [4]:
scenesShotSequence=est.segmentation_speakers(shotSeq,7,4)

In [5]:
len(scenesShotSequence),len(shotSeq)

(74, 637)

In [6]:
sceneShotBoundry=[]
for i in scenesShotSequence:
    sceneShotBoundry.append(shotBondry[i])

start=sceneShotBoundry
start=[0.]+start
del start[-1]
len(start)

74

In [7]:
scenesegFile=scenes_dir+scenesegementedFiles[4]
subtFile=subtitle_dir+subtitleFiles[4]

In [8]:
_,_,sc_start,sc_end=stv.sceneTexts(scenesegFile,subtFile)

In [9]:
sceneShotBoundry[0:10],sc_start[0:3],(sc_end[0:3])

([17.0,
  50.36,
  58.76,
  71.08,
  76.76,
  90.2,
  102.24000000000001,
  107.12,
  113.4,
  240.52],
 [0.0, 6044.0, 112655.0],
 [6044.0, 112655.0, 136963.0])

In [10]:
df=stv.get_text_time(subtFile)

In [11]:
len(df['start_seconds'])

781

In [12]:
df['textList'][1],df['end_second'][1],df['start_seconds'][1]

(u'(Horse whinnies)', 121, 119)

In [13]:
scene_texts,scene_lbls,start,end=stv.sceneTexts(scenesegFile,subtFile)

In [14]:
len(scene_texts),len(scene_lbls),len(start),len(end)

(28, 28, 28, 28)

In [15]:
cl_sc=stv.split_into_sentences(scene_texts[2])

In [16]:
from nltk import sent_tokenize
cl_sc=sent_tokenize(scene_texts[10])

In [17]:
len(scene_texts[10]),len(cl_sc)

(107, 2)

In [18]:
episodeTVM,episodeTVMB,l1=stv.episodTruthValueMan1(scenesegementedFile=scenesegFile,subtitleFile=subtFile)
episodesTVA,episodesTVAB,l2=stv.episodTruthValueShot1(scene_start=start,scene_end=sceneShotBoundry,subtitleFile=subtFile)

In [19]:
len(episodesTVA),len(episodeTVM),l1,l2

(781, 781, 781, 781)

In [20]:
episodesTVAB[25:40],episodesTVA[25:40]

([9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

In [21]:
em.pk(episodeTVM,episodesTVA,k=15,boundary=1)

0.4302477183833116

In [22]:
em.windowdiff(episodeTVM,episodesTVA,k=13,boundary=1)

1.1028645833333333

In [23]:
episodeTVM.count(1)+1,episodesTVA.count(1)+1

(27, 73)

In [24]:
K=(len(episodeTVM)/(2*((episodeTVM.count(1)+1))))

In [25]:
K

14

Computing all evaluations for every different K and C values 

In [26]:
kvalues=[]
cvalues=[]
windiffSHotA=[]
pkShotA=[]
shotRecA=[]
shotPrA=[]
shotRecB=[]
shotPrB=[]
sceneShot=[]
PurityB=[]
NMIAB=[]
Purity=[]
NMIA=[]
Coverage=[]
Coverage1=[]
Coverage2=[]
Coverage3=[]
for k in range(7, 11):
    for c in range(3,9):
        if c<k:
            scenesShotSequence=est.segmentation_speakers(shotSeq,k,c)
            sceneShotBoundry=[]
            for i in scenesShotSequence:
                sceneShotBoundry.append(shotBondry[i])

            start=sceneShotBoundry
            start=[0.]+start
            del start[-1]
            len(start)
            
            episodeTVM,episodeTVMB,l1=stv.episodTruthValueMan1(scenesegementedFile=scenesegFile,subtitleFile=subtFile)
            episodeTVA,episodeTVAB,l2=stv.episodTruthValueShot1(scene_start=start,scene_end=sceneShotBoundry,subtitleFile=subtFile)
            
            
            windiffSHot=round(em.windowdiff(episodeTVM,episodeTVA,k=K,boundary=1),2)
            pkShot=round(em.pk(episodeTVM,episodeTVA,k=K,boundary=1),2)
            
            episodeTVA=[int(i) for i in episodeTVA]
            episodeTVM=[int(i) for i in episodeTVM]
            episodeTVAB=[int(i) for i in episodeTVAB]
            episodeTVMB=[int(i) for i in episodeTVMB]
            episodeTVA=np.array(episodeTVA)
            episodeTVM=np.array(episodeTVM)
            episodeTVAB=np.array(episodeTVAB)
            episodeTVMB=np.array(episodeTVMB)
                
            shotRec=round(em.recall(episodeTVM,episodeTVA),3)
            shotPr=round(em.precision(episodeTVM,episodeTVA),3)
            shotRecb=round(em.recall(episodeTVMB,episodeTVAB,rType='micro'),3)
            shotPrb=round(em.precision(episodeTVMB,episodeTVAB,rType='micro'),3)
            purityb=round(em.purity_score(y_true=episodeTVMB,y_pred=episodeTVAB),3)
            nmib=round(em.nmi(episodeTVMB,episodeTVAB),3)
            purity=round(em.purity_score(y_true=episodeTVM,y_pred=episodeTVA),3)
            nmie=round(em.nmi(episodeTVM,episodeTVA),3)
            #coverage=DiarizationCoverage()
            dc=round(em.coverage(y_true=episodeTVM,y_pred=episodeTVA),3)
            dc1=round(em.coverage(y_true=episodeTVMB,y_pred=episodeTVAB),3)
            dc2=round(em.purity_score(y_true=episodeTVA,y_pred=episodeTVM),3)
            dc3=round(em.purity_score(y_true=episodeTVAB,y_pred=episodeTVMB),3)
                
            
            kvalues.append(k)
            cvalues.append(c)
            windiffSHotA.append(windiffSHot)
            pkShotA.append(pkShot)
            shotRecA.append(shotRec)
            shotRecB.append(shotRecb)
            PurityB.append(purityb)
            NMIAB.append(nmib)
            Purity.append(purity)
            NMIA.append(nmie)
            shotPrA.append(shotPr)
            shotPrB.append(shotPrb)
            sceneShot.append(len(scenesShotSequence))
            Coverage.append(dc)
            Coverage1.append(dc1)
            Coverage2.append(dc2)
            Coverage3.append(dc3)
        else:
            break
print(episodesTVA[0:50])
print(episodeTVM[0:50])
print(episodesTVAB[0:50])
print(episodeTVMB[0:50])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11]
[0 1 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4]


Representing the Evaluation as a table (dataframe)

In [27]:
resulDF=pd.DataFrame()
resulDF['K']=kvalues
resulDF['C']=cvalues
resulDF['wdSH']=windiffSHotA
resulDF['pkSh']=pkShotA
#resulDF['rSh']=recallShot
resulDF['shotRec_01']=shotRecB
resulDF['shotPr_01']=shotPrB
resulDF['shotRec']=shotRecA
resulDF['shotPr']=shotPrA
resulDF['sceneShot']=sceneShot
resulDF['purity_01']=PurityB
resulDF['NMI_01']=NMIAB
resulDF['purity']=Purity
resulDF['NMI']=NMIA
resulDF['Coverage']=Coverage
resulDF['Coverage1']=Coverage1
resulDF['CoverageP']=Coverage2
resulDF['CoverageP1']=Coverage3

In [28]:
resulDF

,K,C,wdSH,pkSh,shotRec_01,shotPr_01,shotRec,shotPr,sceneShot,purity_01,NMI_01,purity,NMI,Coverage,Coverage1,CoverageP,CoverageP1
0,7,3,1.79,0.50,0.004,0.004,0.346,0.084,113,0.843,0.789,0.967,0.020,0.863,0.583,0.863,0.583
1,7,4,1.16,0.43,0.004,0.004,0.192,0.069,74,0.814,0.804,0.967,0.008,0.908,0.672,0.908,0.672
2,7,5,0.94,0.41,0.004,0.004,0.154,0.070,58,0.768,0.806,0.967,0.007,0.927,0.727,0.927,0.727
3,7,6,0.74,0.37,0.004,0.004,0.231,0.133,45,0.767,0.824,0.967,0.032,0.942,0.795,0.942,0.795
4,8,3,1.76,0.51,0.004,0.004,0.346,0.086,111,0.841,0.790,0.967,0.021,0.866,0.585,0.866,0.585
5,8,4,1.16,0.43,0.004,0.004,0.192,0.069,74,0.813,0.803,0.967,0.008,0.908,0.670,0.908,0.670
6,8,5,0.90,0.41,0.004,0.004,0.154,0.073,56,0.768,0.806,0.967,0.007,0.930,0.726,0.930,0.726
7,8,6,0.74,0.38,0.004,0.004,0.231,0.136,44,0.758,0.823,0.967,0.033,0.944,0.791,0.944,0.791
8,8,7,0.77,0.48,0.004,0.004,0.231,0.150,40,0.703,0.788,0.967,0.038,0.949,0.717,0.949,0.717
9,9,3,1.72,0.51,0.004,0.004,0.346,0.087,109,0.841,0.790,0.967,0.022,0.868,0.586,0.868,0.586


In [40]:
resulDF.loc[resulDF['purity'].idxmax()]

K               7.0000
C               3.0000
wdSH            1.7900
pkSh            0.5000
shotRec_01      0.0040
shotPr_01       0.0040
shotRec         0.3460
shotPr          0.0840
sceneShot     113.0000
purity_01       0.8430
NMI_01          0.7890
purity          0.9670
NMI             0.0200
Coverage        0.8630
Coverage1       0.0474
Coverage2       0.8630
Coverage3       0.5826
Name: 0, dtype: float64

In [41]:
resulDF.loc[resulDF['pkSh'].idxmin()]

K              7.0000
C              6.0000
wdSH           0.7400
pkSh           0.3700
shotRec_01     0.0040
shotPr_01      0.0040
shotRec        0.2310
shotPr         0.1330
sceneShot     45.0000
purity_01      0.7670
NMI_01         0.8240
purity         0.9670
NMI            0.0320
Coverage       0.9424
Coverage1      0.1140
Coverage2      0.9424
Coverage3      0.7951
Name: 3, dtype: float64

In [42]:
resulDF.max(axis=None, skipna=None, level=None, numeric_only=None)

K              10.0000
C               8.0000
wdSH            1.7900
pkSh            0.5100
shotRec_01      0.0140
shotPr_01       0.0140
shotRec         0.3460
shotPr          0.1500
sceneShot     113.0000
purity_01       0.8430
NMI_01          0.8240
purity          0.9670
NMI             0.0380
Coverage        0.9590
Coverage1       0.4673
Coverage2       0.9590
Coverage3       0.7951
dtype: float64

In [43]:
labels_true = episodeTVM
labels_true2 = episodeTVMB
#[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,]
labels_pred = episodeTVA
labels_pred2 = episodeTVAB
#[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
 #0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
labels_pred=np.array(labels_pred)
labels_true=np.array(labels_true)
labels_pred2=np.array(labels_pred2)
labels_true2=np.array(labels_true2)

In [44]:
em.purity_score(y_true=labels_true,y_pred=labels_pred),em.purity_score(y_true=labels_true2,y_pred=labels_pred2)

(0.9667093469910372, 0.678617157490397)

In [45]:
em.coverage(y_true=labels_true,y_pred=labels_pred),em.coverage(y_true=labels_true2,y_pred=labels_pred2)

(0.9590268886043534, 0.4673495518565941)

In [46]:
em.purity_score(y_true=labels_pred,y_pred=labels_true),em.purity_score(y_true=labels_pred2,y_pred=labels_true2)

(0.9590268886043534, 0.7708066581306018)

In [36]:
em.purity_score(y_true=labels_pred,y_pred=labels_pred),em.purity_score(y_true=labels_pred2,y_pred=labels_pred2)

(1.0, 1.0)